### Notebook 1 - trade graph description

In [1]:
from networkx.readwrite import json_graph
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import warnings
import json
import os

warnings.filterwarnings('ignore')
year_start = 1988
year_end = 2021
graph_folder = '/Users/koshelev/Documents/lmu/thesis/1-graph_construction/data/preprocessed/trade_graphs/'
weight = 'TradeValue in 1000 USD'
seed = 100

In [2]:
def read_json_file(filename: str) -> nx.Graph:
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)

In [3]:
def graph_algebraic_connectivity(graph: nx.DiGraph, 
                                 is_normalized: bool = False, 
                                 weight_attribute: str = None, 
                                 seed: int = seed) -> float:
    alg_con = nx.linalg.algebraic_connectivity(G=graph, normalized=is_normalized, weight=weight_attribute, 
                                               seed=seed)
    return alg_con


def graph_wiener_index(graph: nx.DiGraph, weight_attribute: str = weight) -> float:
    wiener_index = nx.wiener_index(G=graph, weight=weight_attribute)
    return wiener_index

In [4]:
def graph_descr(graph: nx.DiGraph, weight: str) -> dict:
    output_dict = {}
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    average_degree_w = np.nanmean(list(dict(graph.degree(weight=weight)).values()))
    average_degree_nw = np.nanmean(list(dict(graph.degree(weight=None)).values()))
    average_lcc_nw = nx.average_clustering(G=graph, weight=None)
    average_spl_nw = nx.average_shortest_path_length(G=graph, weight=None)
    algebraic_connect_nw = graph_algebraic_connectivity(graph=graph.to_undirected(), weight_attribute=None)
    average_deg_centr = np.mean(list(nx.degree_centrality(G=graph).values()))
    
    output_dict['num_nodes'] = num_nodes
    output_dict['num_edges'] = num_edges
    output_dict['average_degree_w'] = average_degree_w
    output_dict['average_degree_nw'] = average_degree_nw
    output_dict['average_lcc_nw'] = average_lcc_nw
    output_dict['average_spl_nw'] = average_spl_nw
    output_dict['algebraic_connect_nw'] = algebraic_connect_nw
    output_dict['average_deg_centr'] = average_deg_centr
    
    return output_dict

In [5]:
nodes = []
edges = []
degrees_w = []
degrees_nw = []
alcc_nw = []
spl_nw = []
algebrcon_nw = []
deg_centr = []
years = []

for year in tqdm(range(year_start, year_end + 1)):
    g = read_json_file(graph_folder + f'export_graph_{int(year)}.json')
    print(year)
    years.append(year)
    output_dict = graph_descr(g, weight=weight)
    nodes.append(output_dict['num_nodes'])
    edges.append(output_dict['num_edges'])
    degrees_w.append(output_dict['average_degree_w'])
    degrees_nw.append(output_dict['average_degree_nw'])
    alcc_nw.append(output_dict['average_lcc_nw'])
    spl_nw.append(output_dict['average_spl_nw'])
    algebrcon_nw.append(output_dict['algebraic_connect_nw'])
    deg_centr.append(output_dict['average_deg_centr'])
    
descr_df = pd.DataFrame()
descr_df['nodes'] = nodes
descr_df['edges'] = edges
descr_df['degrees_w'] = degrees_w
descr_df['degrees_nw'] = degrees_nw
descr_df['alcc_nw'] = alcc_nw
descr_df['spl_nw'] = spl_nw
descr_df['algebrcon_nw'] = algebrcon_nw
descr_df['deg_centr'] = deg_centr
descr_df.index = years

  0%|          | 0/34 [00:00<?, ?it/s]

1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [6]:
descr_df.to_csv('graph_descriptions/trade.csv', index=True)

In [7]:
descr_df.head()

,nodes,edges,degrees_w,degrees_nw,alcc_nw,spl_nw,algebrcon_nw,deg_centr
1988,201,4791,1.733868e+07,47.671642,0.859048,0.239403,0.994325,0.238358
1989,201,6546,2.361175e+07,65.134328,0.823337,0.345075,0.993742,0.325672
1990,202,7860,2.716548e+07,77.821782,0.812359,0.450643,0.998835,0.387173
1991,199,8336,2.884452e+07,83.778894,0.802795,0.502462,1.995073,0.423126
1992,221,10528,2.942381e+07,95.276018,0.796802,0.563266,0.993622,0.433073
